In [1]:
import os
import asyncio
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

from extend_ai_toolkit.langchain.toolkit import ExtendLangChainToolkit
from extend_ai_toolkit.shared import Configuration, Scope, Product, Actions

from IPython.display import display, clear_output
import ipywidgets as widgets

import nest_asyncio
nest_asyncio.apply()

In [2]:
# Load environment variables
load_dotenv()

# Get required environment variables
api_key = os.environ.get("EXTEND_API_KEY")
api_secret = os.environ.get("EXTEND_API_SECRET")

# Validate environment variables
if not all([api_key, api_secret]):
    raise ValueError("Missing required environment variables. Please set EXTEND_API_KEY and  EXTEND_API_SECRET")

In [3]:
llm = ChatOpenAI(
    model="gpt-4o",
)

extend_langchain_toolkit = ExtendLangChainToolkit.default_instance(
    api_key,
    api_secret,
    Configuration(
      scope=[
        Scope(Product.VIRTUAL_CARDS, actions=Actions(read=True,update=True)),
        Scope(Product.CREDIT_CARDS, actions=Actions(read=True)),
        Scope(Product.TRANSACTIONS, actions=Actions(read=True,update=True)),
        Scope(Product.EXPENSE_CATEGORIES, actions=Actions(read=True)),
        Scope(Product.RECEIPT_ATTACHMENTS, actions=Actions(read=True)),
       ]
    )
)

tools = []
tools.extend(extend_langchain_toolkit.get_tools())

# Create the react agent
langgraph_agent_executor = create_react_agent(
    llm,
    tools
)

In [8]:
async def chat_with_agent():
    print("\nWelcome to the Extend AI Assistant!")
    print("I'll send a sequence of predefined messages to the agent and show the responses.\n")
    
    # Define the messages to send
    messages = [
        "please show me transactions from feb 2025 from langchain inc",
        "please tell me about my expense categories and labels",
        "please tell me which transactions from feb 2025 are missing receipts"    ]
    
    # Send each message and print the response
    for i, user_input in enumerate(messages, 1):
        print(f"\n--- Message {i}: {user_input} ---")
        print("Assistant: ", end="", flush=True)
        
        # Process the query
        result = await langgraph_agent_executor.ainvoke({
            "input": user_input,
            "messages": [
                SystemMessage(content="You are a helpful assistant that can interact with the Extend API to manage virtual cards, credit cards, and transactions."),
                HumanMessage(content=user_input)
            ]
        })
        
        # Extract and print the assistant's message
        for message in result.get('messages', []):
            if isinstance(message, AIMessage):
                print(message.content)
        
        # Add a separator between messages
        print("\n" + "-" * 50)
    
    print("\nAll messages have been processed.")

In [9]:
await chat_with_agent()


Welcome to the Extend AI Assistant!
I'll send a sequence of predefined messages to the agent and show the responses.


--- Message 1: please show me transactions from feb 2025 from langchain inc ---
Assistant: 
Here are the transactions from February 2025 with Langchain Inc.:

1. **Transaction ID:** txn_2ahU0UDCn6e7CwnjKEMP24
   - **Amount:** $39.00
   - **Status:** CLEARED
   - **Cardholder Name:** Jonathan Bailey
   - **Recipient Name:** Jonathan Bailey
   - **Merchant:** LANGCHAIN, INC.
   - **Merchant Category Code (MCC):** Computer Software Stores
   - **Review Status:** PENDING_REVIEW
   - **Receipt Required:** No
   - **Receipt Attachments Count:** 0
   - **Synced to ERP:** No

2. **Transaction ID:** txn_2mxwPYd582B9TtzQlI7Sd9
   - **Amount:** $39.00
   - **Status:** CLEARED
   - **Cardholder Name:** Jonathan Bailey
   - **Recipient Name:** Jonathan Bailey
   - **Merchant:** LANGCHAIN, INC.
   - **Merchant Category Code (MCC):** Computer Software Stores
   - **Review Status:** 